# CSE-CIC-IDS 2017 K Nearest Neighbors Classifier

In [1]:
model_id = "knn3-featureselection"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import glob

In [3]:
NOTEBOOK_PATH = "C:/Users/Xetrov/Desktop/SciFair20/Code/"

In [4]:
selected_features = [
    'Destination Port',
    'Init_Win_bytes_forward',
    'Avg Fwd Segment Size',
    'Subflow Fwd Bytes',
    'Packet Length Std',
    'Packet Length Variance',
    'Fwd Packet Length Mean',
    'Init_Win_bytes_backward',
    'Fwd Packet Length Max',
    'Packet Length Mean',
    'Total Length of Fwd Packets',
    'Average Packet Size',
    'Max Packet Length'
]

In [5]:
x_scaled = pd.read_csv(NOTEBOOK_PATH + "IDS2017/x_scaled_powertransform.csv")[selected_features]

y_df_enc = pd.read_csv(NOTEBOOK_PATH + "IDS2017/y_all_binary.csv")

# Split data into train and test

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_valtest, y_train, y_valtest = train_test_split(x_scaled, y_df_enc, test_size = 0.4, random_state = 42)

In [8]:
x_val, x_test, y_val, y_test = train_test_split(x_valtest, y_valtest, test_size = 0.5, random_state = 42)

In [9]:
del x_train
del y_train
del x_valtest 
del y_valtest

# ADASYN

In [10]:
x_train_res = pd.read_csv(NOTEBOOK_PATH + "IDS2017/x_adasyn_binary.csv")[selected_features]

In [11]:
y_train_res = pd.read_csv(NOTEBOOK_PATH + "IDS2017/y_adasyn_binary.csv")['IsAttack']

# Train model

In [12]:
from sklearn.neighbors import KNeighborsClassifier

In [13]:
%%time
clf = KNeighborsClassifier(n_neighbors=5, weights='distance', n_jobs=12).fit(x_train_res, y_train_res)

Wall time: 46min 5s


**Save model**

In [14]:
import pickle

In [15]:
import time

model_filename = model_id  + " [" + time.strftime("%Y%m%d %H%M") + "]"

In [16]:
save_file = open(NOTEBOOK_PATH + "Models/" + model_filename + ".pkl", "wb")
saved_model = pickle.dump(clf, save_file)
save_file.close()

**Test model**

In [17]:
pred = clf.predict(x_val)

In [18]:
pred_series = pd.Series(pred, name="Pred").replace({0: 'Benign', 1: 'Attack'})
y_series = pd.Series(y_val.to_numpy().ravel(), name="Actual").replace({0: 'Benign', 1: 'Attack'})

matrix = pd.crosstab(pred_series, y_series, margins=True)
matrix

Actual,Attack,Benign,All
Pred,,,
Attack,111481,898,112379
Benign,461,453309,453770
All,111942,454207,566149


# Test Set

In [19]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [20]:
pred = clf.predict(x_test)
y_test_npy = y_test.to_numpy().ravel()

precision = precision_score(y_test_npy, pred)
print("Precision:", precision)

recall = recall_score(y_test_npy, pred)
print("Recall:", recall)

f1 = f1_score(y_test_npy, pred)
print("F1:", f1)

Precision: 0.9921254205741284
Recall: 0.9956267567057894
F1: 0.9938730049168351


In [21]:
pred_series = pd.Series(pred, name="Pred").replace({0: 'Benign', 1: 'Attack'})
y_series = pd.Series(y_test.to_numpy().ravel(), name="Actual").replace({0: 'Benign', 1: 'Attack'})

matrix = pd.crosstab(pred_series, y_series, margins=True)
matrix

Actual,Attack,Benign,All
Pred,,,
Attack,110872,880,111752
Benign,487,453910,454397
All,111359,454790,566149


# Log results

In [18]:
model_log = open(NOTEBOOK_PATH + "model_log.txt", "a")

model_log.write("\n" + model_filename)
model_log.write("\n\tF1 Micro: " + str(f1_micro))
model_log.write("\n\tF1 Macro: " + str(f1_macro))

model_log.close()